# Compare results of solvers 

## import library

In [1]:
import os
import re
import subprocess
import time
import datetime as dt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from itertools import permutations
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
input_file_path
distance_file_path
time_file_path
output_file_path
vehicle_setting_file_path

NameError: name 'input_file_path' is not defined

In [8]:
def find_csv_file(file_name, directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith("input.csv"):
                current_directory = os.getcwd()  # 現在のディレクトリを取得
                relative_path = os.path.relpath(os.path.join(root, file), current_directory)  # 指定したディレクトリから現在のディレクトリまでの相対パスを取得
                return relative_path  # 相対パスを返す

directory_path = "../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001"  # ディレクトリのパスを適切なものに置き換える
input_file_path = find_input_csv_file(directory_path)
input_file_path

'../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001/request_haisha-solver-input.csv'

##  ファイル内検索 + ファイルパス取得

In [19]:
class FileFinder:
    def __init__(self, directory, file_name):
        self.directory = directory
        self.file_name = file_name

    def find_csv_file(self):
        for root, dirs, files in os.walk(self.directory):
            for file in files:
                if file.endswith(f"{self.file_name}.csv"):
                    current_directory = os.getcwd()  # 現在のディレクトリを取得
                    relative_path = os.path.relpath(os.path.join(root, file), current_directory)  # 指定したディレクトリから現在のディレクトリまでの相対パスを取得
                    return relative_path  # 相対パスを返す
                    # TODO:保存先のディレクトリ指定

### クラステスト

In [38]:
directory = "../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001"
file_name = "input"

finder = FileFinder(directory, file_name)
result = finder.find_csv_file()
result


'../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001/request_haisha-solver-input.csv'

## ヘテロ車両定義ファイル・ファイルパス作成

In [39]:
headers = ['VEHICLENO.', 'LOAD_LIMIT', 'E-CAPACITY', 'E-INITIAL', 'E-COST', 'E-MARGIN', 'VTYPE', 'ETYPE']
vehicle_setting_df = pd.DataFrame(columns=headers)
# TODO: 重量の順番を考慮して計算
# LOAD_LIMITが13tである5行のデータを作成
vehicle_data_13t = {
    'VEHICLENO.': range(0, 5),
    'LOAD_LIMIT': [12800] * len(range(0, 5)),
    'E-CAPACITY': [None] * len(range(0, 5)),
    'E-INITIAL': [None] * len(range(0, 5)),
    'E-COST': [None] * len(range(0, 5)),
    'E-MARGIN': [None] * len(range(0, 5)),
    'VTYPE': [None] * len(range(0, 5)),
    'ETYPE': [None] * len(range(0, 5)),
}

# LOAD_LIMITが8tである10行のデータを作成
vehicle_data_8t = {
    'VEHICLENO.': range(5, 15),
    'LOAD_LIMIT': [7800] * 10,
    'E-CAPACITY': [None] * 10,
    'E-INITIAL': [None] * 10,
    'E-COST': [None] * 10,
    'E-MARGIN': [None] * 10,
    'VTYPE': [None] * 10,
    'ETYPE': [None] * 10,
    'LOAD_LIMIT2': [None] * 10
}

# LOAD_LIMITが4tである10行のデータを作成
vehicle_data_4t = {
    'VEHICLENO.': range(15, 25),
    'LOAD_LIMIT': [3800] * 10,
    'E-CAPACITY': [None] * 10,
    'E-INITIAL': [None] * 10,
    'E-COST': [None] * 10,
    'E-MARGIN': [None] * 10,
    'VTYPE': [None] * 10,
    'ETYPE': [None] * 10,
    'LOAD_LIMIT2': [None] * 10
}

# LOAD_LIMITが3tである15行のデータを作成
vehicle_data_3t = {
    'VEHICLENO.': range(25, 40),
    'LOAD_LIMIT': [2800] * 15,
    'E-CAPACITY': [None] * 15,
    'E-INITIAL': [None] * 15,
    'E-COST': [None] * 15,
    'E-MARGIN': [None] * 15,
    'VTYPE': [None] * 15,
    'ETYPE': [None] * 15,
    'LOAD_LIMIT2': [None] * 15
}
# 4つのデータセットをDataFrameに変換し、結合
vehicle_df_13t = pd.DataFrame(vehicle_data_13t, columns=headers)
vehicle_df_8t = pd.DataFrame(vehicle_data_8t, columns=headers)
vehicle_df_4t = pd.DataFrame(vehicle_data_4t, columns=headers)
vehicle_df_3t = pd.DataFrame(vehicle_data_3t, columns=headers)

new_rows = pd.concat([vehicle_df_13t, vehicle_df_8t, vehicle_df_4t, vehicle_df_3t], ignore_index=True)
vehicle_setting_df = pd.concat([vehicle_setting_df, new_rows], ignore_index=True)
vehicle_setting_df

,VEHICLENO.,LOAD_LIMIT,E-CAPACITY,E-INITIAL,E-COST,E-MARGIN,VTYPE,ETYPE
0,0,12800,None,None,None,None,None,None
1,1,12800,None,None,None,None,None,None
2,2,12800,None,None,None,None,None,None
3,3,12800,None,None,None,None,None,None
4,4,12800,None,None,None,None,None,None
5,5,7800,None,None,None,None,None,None
6,6,7800,None,None,None,None,None,None
7,7,7800,None,None,None,None,None,None
8,8,7800,None,None,None,None,None,None
9,9,7800,None,None,None,None,None,None


In [ ]:
directory = "../../tests/kouei_data/検証用データ1/0530データ-20240610-135109304-TES1EDI001-TES1AP001"
input_file_path = FileFinder(directory, "input").find_csv_file()
distance_file_path = FileFinder(directory, "dist").find_csv_file()
time_file_path = FileFinder(directory, "time").find_csv_file()
vs_file_path = FileFinder(directory, "vs").find_csv_file()
output_file_path = directory

In [23]:
# TODO:ヘテロ車両定義ファイルを作成する
class vehicles_setting:
    


# CNS Solver

In [ ]:
std_out = subprocess.check_output([
        "pypy","../../CNSsolver_20230602_v2303/vrp_classical/code/main_tw.py",
        "-i", input_file_path,
        "-d", distance_file_path,
        "-t", time_file_path,
        "-o", output_file_path,
        "-v",  40, 
        "-vs", vs_file_path,
        "-rv", # reduce vehicle
        "-s", 6:00,
        "-e", 16:30,
        "-mt", #multi-trip (rotation)
        # "-dem", str(capacity),
        "-lpt", "hard", # hard constraint on package weight v.s. vehicle capacity
        #"-avedistper", "10"
        # "-l" #1.20.2022
        # "-multithread", "0"
        "ls",
        "-l",
        ]  # , text=True
    )
# 標準出力をbytes型で受け取るには下記のコードを追加する
# (['ls', '-l'])
# 上記にtext=Trueを付けるとstr型になる

# save std_out as txt file
with open(
        f"results/simulation/{cust_name}/{instance_name}_log_CARG.txt", "wb"
    ) as file:
        file.write(std_out)